In [1]:
from langchain_community.embeddings.huggingface import HuggingFaceInstructEmbeddings
from langchain_community.llms.huggingface_hub import HuggingFaceHub
from langchain_community.vectorstores.faiss import FAISS
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.chains import StuffDocumentsChain, LLMChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser



In [2]:
text = ""

with open('./ANLP Assignment2 update 2.pdf', 'rb') as f:
    pdf_reader = PdfReader(f)
    for page in pdf_reader.pages:
        text += page.extract_text()
        
text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=800,
        chunk_overlap=400,
        length_function=len
    )

chunks = text_splitter.split_text(text)

# Embed
model_kwargs = {'device': 'cpu'}
eb_model = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", model_kwargs=model_kwargs)
# eb_model = AutoModel.from_pretrained('./instructor-xl')
vectorstore = FAISS.from_texts(texts=chunks, embedding=eb_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

/media/james/2b44c141-eec6-4c63-a888-30e9ac5660bd/git/NLP_Assignment_2/.venv/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [3]:
# Instantiate ConversationBufferMemory
memory = ConversationBufferWindowMemory(k=2)

gen_llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":1000})

/media/james/2b44c141-eec6-4c63-a888-30e9ac5660bd/git/NLP_Assignment_2/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [ ]:
conversation = ConversationChain(llm=gen_llm, verbose=True, memory=memory )

In [ ]:
conversation.predict(input='HI')

In [ ]:
conversation.predict(input='What is your name?')


In [ ]:
conversation.predict(input='Ny name is James')


In [ ]:
conversation.predict(input='What is my name?')


In [ ]:
conversation.predict(input='This is context: The assignment weights is 30 percent coding and 70 persent report. Using that context to answer my question: What is the weight of this assignment?')


In [ ]:
conversation.predict(input='Can you answer longer using natural language?')


In [ ]:
conversation.memory.buffer

In [55]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is. But the most important thing, you have to answer using natural language as a human.{chat_history}"""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
contextualize_q_chain = contextualize_q_prompt | gen_llm | StrOutputParser()

In [56]:


qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. But the most important thing, you have to answer using natural language as a human.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]


rag_chain = (
    RunnablePassthrough.assign(
        context=contextualized_question | retriever | format_docs
    )
    | qa_prompt
    | gen_llm
)


In [57]:
chat_history = []

question = "What is the weight of this assignment?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg])

second_question = "How many of them is from report?"
ai_msg = rag_chain.invoke({"question": second_question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=second_question), ai_msg])


In [58]:
chat_history

[HumanMessage(content='What is the weight of this assignment?'),
 '30% of this assignment weigh ng is for the code, and 70% for the report. The code and the report are marked per rubric.',
 HumanMessage(content='How many of them is from report?'),
 '70%']

In [59]:
second_question = "What is this?"
ai_msg = rag_chain.invoke({"question": second_question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=second_question), ai_msg])


In [60]:
chat_history

[HumanMessage(content='What is the weight of this assignment?'),
 '30% of this assignment weigh ng is for the code, and 70% for the report. The code and the report are marked per rubric.',
 HumanMessage(content='How many of them is from report?'),
 '70%',
 HumanMessage(content='What is this?'),
 'The purpose of this assignment is to provide you with the opportunity to implement a real -world solu on for an NLP problem. This assignment will support you in achieving the following Course Learning Outcomes: 2. Use exis ng natural language processing tools to conduct basic natural language processing, such as text normaliza on, named en ty extrac on, or syntac c parsing. 3. Use machine learning tools to build sol']

In [62]:
second_question = "Who is the president of USA?"
ai_msg = rag_chain.invoke({"question": second_question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=second_question), ai_msg])


HfHubHTTPError: 422 Client Error: Unprocessable Entity for url: https://api-inference.huggingface.co/models/google/flan-t5-xxl (Request ID: cCKzdmCjEL79G2PC2Evme)

Input validation error: `inputs` must have less than 1024 tokens. Given: 1250
Make sure 'text2text-generation' task is supported by the model.

In [ ]:
chat_history